# Labeling of Measurements and Label Propagation

MVG comes with a labeling API that allows a user to store important information about a measurement.
There are multiple reasons why you would want to label measurements and some of the benefits include

1.  Additional information about modes. For example, a mode with multiple critical measurements is likely a failure mode
2.  Improved mode detection
3.  As personal notes for keeping track of interesting events in the data

In addition, the LabelPropagation feature enables to propagate those labels to the measurements that do not contain any labels.

This example will go through what a label is in MVG, how the labeling API functions, and how to utilize the LabelPropagation feature.

### Prerequisites

To follow this example you need to have

1. Installed `mvg` package
2. A token for API access from Viking Analytics
3. The database needs to be populated with our example assets. This can be achieved by running the ["Sources and Measurement"](2-sources_and_measurements.ipynb) example.
4. Use of the LabelPropagation feature requires the results from ModeId. We can use the results available after running the ["Analysis and Results Visualization"](6-analysis_visual.ipynb) example.

## Import the required packages and functions

In [ ]:
import os
from mvg import MVG
from mvg.analysis_classes import parse_results
from mvg.plotting import plot_labels_over_time

## Create a session for API access

Note that the `TOKEN` is used both for authorization and authentication. Thus, each unique token represents a unique user and each user has their own unique database on the Viking Analytics MultiViz Vibration service.

**You need to insert your token received from Viking Analytics here:**

In [ ]:
TOKEN = os.environ["TEST_TOKEN"]
URL = "https://api.beta.multiviz.com"
session = MVG(URL, TOKEN)

session.check_version()

### Retrieve ModeId results

Once the API session is live, we start by checking if the source `u0001` we will use is available in the database.

In [ ]:
SOURCE_ID = "u0001"
session.get_source(SOURCE_ID)

Now that we know that the source exists we need to check its measurements to know which ones to label.

In this example, the ModeId results are used to select the measurements to label and to compare with the results from the LabelPropagation feature.
The ModeId results are NOT needed to add a label to the measurements.

In [ ]:
ModeId_request_id = "5649faa234642e7207fd9800481bcb6e"
raw_result = session.get_analysis_results(ModeId_request_id)
result = parse_results(raw_result, "Europe/Stockholm", "s")
u0001_df = result.to_df()
u0001_df.head()

In [ ]:
result.plot()


## Adding labels

A label in MVG relates to a specific measurement, which are identifiable by the source ID and the timestamp. Each measurement can only have one label. The label itself has three components

1. A label identification string
2. A severity level number
3. And notes

.. note:: The label identification string is unique for each label and is case and spelling sensitive, i.e. `"failure"` and `"Failure"` are not interpreted as the same label. The severity level is an integer where a larger number indicates a more severe problem. They do not need to be the same for different instances of the same label. The notes exist for the end user to add extra information to a certain label and is not used by MVG in any way.

For this example, let us say that there was a bearing failure of the machine in the first measurement of the source, so we will add a label to the afflicted measurements. To do so, we use the `MVG.create_label()` method.

In [ ]:
first_measurement = 1570273260
session.create_label(
    sid=SOURCE_ID,
    timestamp=1570273260,
    label="bearing_failure",
    severity=3,
    notes="Imaginary bearing failure"
)

It might be a good idea to label more than one measurement per source.
Therefore, we proceed to attach a label to the top 3 measurements with the largest mode probability of mode from the ModeId feature results.
First, we create a dataframe to identify these measurements.

In [ ]:
Top3meas_df = u0001_df[u0001_df.groupby('labels')['mode_probability'].rank(method='dense', ascending=False) <= 3]
Top3meas_df

There are three modes on the ModeId results.
We will label measurements from mode 0 as "Healthy", measurements from mode 1 as "Transition", measurements from mode 2 as "Faulty".

In [ ]:
modes = Top3meas_df['labels'].unique()
mode_dfs = {mode: Top3meas_df[Top3meas_df['labels'] == mode] for mode in modes}

for mode in mode_dfs:
    if mode == 0:
        label_to_add = "Healthy"
    elif mode == 1:
        label_to_add = "Transition"
    else:
        label_to_add = "Faulty"
    case_df = mode_dfs[mode]

    for index, row in case_df.iterrows():
        session.create_label(
            sid=SOURCE_ID,
            timestamp=row['timestamps'],
            label=label_to_add,
            severity=int(mode)+1,
            notes="This is a demo."
        )


## Reading Labels

To get a list of all the labels for a source we use the `MVG.list_labels()` method.

In [ ]:
session.list_labels(SOURCE_ID)

As you can see, the measurements that we just added labels show up. To inspect the label of a specific measurement we use `MVG.get_label()`

In [ ]:
session.get_label(SOURCE_ID, first_measurement)

## Updating labels

Now we realize that the label we had in the first measurement correspond to mode 0 and the label that we originally added is incorrect.
Then we use the `session.update_label()` method to change the label.

In [ ]:

session.update_label(
    sid=SOURCE_ID,
    timestamp=first_measurement,
    label="healthy",
    severity=0,
    notes="This was an OK measurement"
)

# List the labels again
labels = session.list_labels(SOURCE_ID)
labels

## Plotting labels

To get an overview of the data and labels it can be very good to plot the labels over time. The `plotting` module of `mvg` contains the function `plot_labels_over_time` that does just that. The plot is very similar to that of the ModeId plots, so it is easy to compare results. We can add the `include_unlabeled=True` keyword to plot the unlabeled measurements as well.

In [ ]:
labels = session.list_labels(SOURCE_ID, include_unlabeled=True)
plot_labels_over_time(labels, SOURCE_ID, timeunit="s")

As you can see we only labeled a small subset of the data. The idea is not to label all the measurements, but to label a part and then use that information to make inference about the unlabeled measurements. 

## Deleting labels

Now, we check the total number of existing labels in our source.

In [ ]:
labels = session.list_labels(SOURCE_ID)
print(f"Currently, there are {len(labels)} labels")

To delete labels we use the `MVG.delete_label()` method.
Let's use that to remove the label from the first measurement and keep only three labels per mode.

In [ ]:
session.delete_label(SOURCE_ID, first_measurement)

In [ ]:
labels = session.list_labels(SOURCE_ID)
print(f"Currently, there are {len(labels)} labels")

## Label Propagation

Until this point, we had added labels to some of our measurements but we might be interested on adding the labels to the rest of the measurements.
We use the ["LabelPropagation"](../features/label_propagation.md) feature to achieve this.

The LabelPropagation feature uses the results from the ModeId feature to propagate the labels.
However, the labels are attached to each measurement individually and not to the modes as a whole.

The use the LabelPropagation feature, we need to know which source is to be used and the `request_id` that holds the results of the ModeId feature.
Then, we can request the analysis.

In [ ]:
ModeId_request_id = "5649faa234642e7207fd9800481bcb6e"

In [ ]:
params = {"model_ref": ModeId_request_id}
labelProp_u0001 = session.request_analysis(SOURCE_ID, "LabelPropagation", params)
labelProp_request_id = labelProp_u0001["request_id"]
labelProp_u0001

In [ ]:
out_status =  session.get_analysis_status("a36badb8f2788c247ae193acb3eaeb3e")
#out_status =  session.get_analysis_status("5649faa234642e7207fd9800481bcb6e")
out_status

Before we are able to get the analysis results, we need to wait until those analyses are successfully completed.
Once the analysis is complete, one gets the results by calling the corresponding “request_id” for the LabelPropagation analysis.
The raw results can be retrieved by calling the `get_analysis_results` function.

In [ ]:
labelprop_output = session.get_analysis_results(request_id=labelProp_request_id)
labelprop_output.keys()

The output is a dictionary that contains seven key elements.
These elements are:
- `"status"` indicates if the analysis was successful.
- `"request_id"` is the identifier of the requested analysis.
- `"feature"` is the name of the request feature.
- `"results"` includes the numeric results.
- `"inputs"` includes the input information for the request analysis.
- `"error_info"` includes the error information in case the analysis fails and it is empty if the analysis is successful.
- `"debug_info"` includes debugging (log) information related to the failed analysis.

To make the results more accessible, we’ll use the analysis_classes.
The `parse_results` function will take the raw_results of (any) analysis and represent them in a python object with a number of convenience methods for plotting and exporting.
The parse function will automatically determine the kind (feature) of analysis based on the raw_results and based on the defined timezone convert the epoch into actual dates.

In [ ]:
labelprop_results = parse_results(labelprop_output, "Europe/Stockholm", "s")


Once we had parsed the results, we can export them to a dataframe for ease of manipulation

In [ ]:
labelprop_results.to_df()


We can also display the results and visualize the labels given to the different measurements.

In [ ]:
labelprop_results.plot()

As you can see from the results BLABLABLA